<a href="https://colab.research.google.com/github/gargayush792/DL_project/blob/achoudhary40/Pretrained_Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# !pip install inltk
!pip install fasttext
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.hi.zip
!unzip 'wiki.hi.zip'

     |████████████████████████████████| 68 kB 3.1 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3139662 sha256=f9c43c877dd620530866ae7ceb88a9f6aed0013f61ce66992e76306f954253cf
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [7]:
import pandas as pd
import numpy as np

In [46]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses

In [9]:
# from inltk.inltk import get_embedding_vectors
# from inltk.inltk import setup
# setup("hi")

### Dataset reading and annotations

In [10]:
dataset = pd.read_csv('Bhaav-Dataset.csv')

In [11]:
dataset

,Sentences,Annotation
0,रमजान के पूरे तीस रोजों के बाद ईद आयी है,1
1,"कितना मनोहर, कितना सुहावना प्रभाव है",1
2,"वृक्षों पर अजीब हरियाली है, खेतों में कुछ अजीब...",1
3,"आज का सूर्य देखो, कितना प्यारा, कितना शीतल है,...",1
4,गाँव में कितनी हलचल है,1
...,...,...
20299,फिर यहाँ सर्दी हो जाती है,4
20300,दिन-भर मैं यह देखती रहती हूँ कि धूप का टुकड़ा क...,4
20301,"पार्क का कोई ऐसा कोना नहीं, जहाँ मैं घड़ी-आधा घ...",4
20302,लेकिन यह बेंच मुझे सबसे अच्छी लगती है,1


In [12]:
dataset['Annotation'].value_counts()

4    11697
2     3168
1     2463
3     1512
0     1464
Name: Annotation, dtype: int64

In [13]:
dataset['Emotion'] = np.where(
    dataset['Annotation'] == 0, 'Anger', np.where(
        dataset['Annotation'] == 1, 'Joy', np.where(
            dataset['Annotation'] == 2, 'Sad', np.where(
                dataset['Annotation'] == 3, 'Suspense', 'Neutral'))))

In [14]:
dataset['Emotion'].value_counts()

Neutral     11697
Sad          3168
Joy          2463
Suspense     1512
Anger        1464
Name: Emotion, dtype: int64

### Convert to TF Dataset and Train/Test Split

In [15]:
full_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(dataset['Sentences'].values, tf.string),
            tf.cast(dataset['Annotation'].values, tf.int32)
        )
    )
)

In [16]:
train_size = int(dataset.shape[0]*0.8)

In [17]:
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)

In [18]:
batch_size = 16

In [19]:
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

### Preprocessing
* Tokenisation
* Punctuation Removal
* Embed words to ints

In [20]:
max_features = 10000
sequence_length = 100

vectorize_layer = layers.TextVectorization(
    standardize="strip_punctuation",
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [21]:
train_text = train_dataset.map(lambda x, y: x)

In [22]:
vectorize_layer.adapt(train_text)

In [23]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [24]:
text_batch, label_batch = next(iter(train_dataset))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", first_label)
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b'\xe0\xa4\xb0\xe0\xa4\xae\xe0\xa4\x9c\xe0\xa4\xbe\xe0\xa4\xa8 \xe0\xa4\x95\xe0\xa5\x87 \xe0\xa4\xaa\xe0\xa5\x82\xe0\xa4\xb0\xe0\xa5\x87 \xe0\xa4\xa4\xe0\xa5\x80\xe0\xa4\xb8 \xe0\xa4\xb0\xe0\xa5\x8b\xe0\xa4\x9c\xe0\xa5\x8b\xe0\xa4\x82 \xe0\xa4\x95\xe0\xa5\x87 \xe0\xa4\xac\xe0\xa4\xbe\xe0\xa4\xa6 \xe0\xa4\x88\xe0\xa4\xa6 \xe0\xa4\x86\xe0\xa4\xaf\xe0\xa5\x80 \xe0\xa4\xb9\xe0\xa5\x88', shape=(), dtype=string)
Label tf.Tensor(1, shape=(), dtype=int32)
Vectorized review (<tf.Tensor: shape=(1, 100), dtype=int64, numpy=array([[   1,    2,  839, 2833, ...,    0,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)


In [25]:
train_ds = train_dataset.map(vectorize_text)
val_ds = test_dataset.map(vectorize_text)

In [43]:
vocab = vectorize_layer.get_vocabulary()
word_index = dict(zip(vocab, range(len(vocab))))

In [41]:
import fasttext
import fasttext.util
ft = fasttext.load_model('wiki.hi.bin')
word = "नृत्य"
print("Embedding Shape is {}".format(ft.get_word_vector(word).shape))
print("Nearest Neighbors to {} are:".format(word))
ft.get_nearest_neighbors(word) 

Embedding Shape is (300,)
Nearest Neighbors to नृत्य are:


[(0.8913929462432861, 'नृत्य।'),
 (0.8440190553665161, 'नृत्यगान'),
 (0.8374733924865723, 'नृत्यगीत'),
 (0.8336297869682312, 'नृत्यों'),
 (0.8265783190727234, 'नृत्यरत'),
 (0.7971948385238647, 'नृत्यकला'),
 (0.7879464626312256, 'नृत्त'),
 (0.7682990431785583, 'नृतक'),
 (0.7622954845428467, 'नृत्यरचना'),
 (0.7602956295013428, 'नृत्यग्राम')]

In [44]:
embedding_dim = 300

# Prepare embedding matrix
hits = 0
misses = 0
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in word_index.items():
    embedding_vector = ft.get_word_vector(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 10000 words (0 misses)


### Model definition

In [51]:
model = tf.keras.Sequential([
  layers.Embedding(max_features, embedding_dim, embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix), trainable=False),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(5, activation='softmax')])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 300)         3000000   
                                                                 
 dropout_4 (Dropout)         (None, None, 300)         0         
                                                                 
 global_average_pooling1d_2   (None, 300)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_5 (Dropout)         (None, 300)               0         
                                                                 
 dense_2 (Dense)             (None, 5)                 1505      
                                                                 
Total params: 3,001,505
Trainable params: 1,505
Non-trainable params: 3,000,000
________________________________________

In [52]:
model.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=tf.metrics.SparseCategoricalAccuracy())

In [53]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10
1016/1016 [==============================] - 11s 9ms/step - loss: 1.3620 - sparse_categorical_accuracy: 0.5510 - val_loss: 1.1563 - val_sparse_categorical_accuracy: 0.6646
Epoch 2/10
1016/1016 [==============================] - 9s 9ms/step - loss: 1.3086 - sparse_categorical_accuracy: 0.5540 - val_loss: 1.1256 - val_sparse_categorical_accuracy: 0.6646
Epoch 3/10
1016/1016 [==============================] - 9s 9ms/step - loss: 1.2899 - sparse_categorical_accuracy: 0.5540 - val_loss: 1.1056 - val_sparse_categorical_accuracy: 0.6646
Epoch 4/10
1016/1016 [==============================] - 9s 9ms/step - loss: 1.2763 - sparse_categorical_accuracy: 0.5539 - val_loss: 1.0917 - val_sparse_categorical_accuracy: 0.6646
Epoch 5/10
1016/1016 [==============================] - 9s 9ms/step - loss: 1.2665 - sparse_categorical_accuracy: 0.5535 - val_loss: 1.0817 - val_sparse_categorical_accuracy: 0.6646
Epoch 6/10
1016/1016 [==============================] - 9s 9ms/step - loss: 1.2591 - spar

In [54]:
val_softmax = model.predict(val_ds)

In [55]:
val_labels = np.argmax(val_softmax, axis=1)

In [56]:
val_pd = dataset[train_size:]

In [57]:
val_pd['pred_labels'] = val_labels

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [58]:
val_pd

,Sentences,Annotation,Emotion,pred_labels
16243,हमारे पास एक मोटा ताजा ऊंट है और हम भूखे मर रह...,2,Sad,4
16244,"चीते ने ठंडी सांस भरी, 'क्या करें",4,Neutral,4
16245,शेर ने उसे अभयदान जो दे रखा है,4,Neutral,4
16246,देखो तो ऊंट की पीठ का कूबड़ कितना बड़ा हो गया है,4,Neutral,4
16247,चर्बी ही चर्बी भरी है इसमें,4,Neutral,4
...,...,...,...,...
20299,फिर यहाँ सर्दी हो जाती है,4,Neutral,4
20300,दिन-भर मैं यह देखती रहती हूँ कि धूप का टुकड़ा क...,4,Neutral,4
20301,"पार्क का कोई ऐसा कोना नहीं, जहाँ मैं घड़ी-आधा घ...",4,Neutral,4
20302,लेकिन यह बेंच मुझे सबसे अच्छी लगती है,1,Joy,4


In [59]:
val_pd['pred_labels'].value_counts()

4    4061
Name: pred_labels, dtype: int64

In [60]:
val_pd['Annotation'].value_counts()

4    2699
2     625
1     416
0     183
3     138
Name: Annotation, dtype: int64

### LSTM

In [61]:
model = tf.keras.Sequential([
    layers.Embedding(max_features, embedding_dim, mask_zero=True, embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix), trainable=False),
    tf.keras.layers.LSTM(32),
    layers.Dense(16, activation='relu'),
    layers.Dense(5)
])

In [62]:
model.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=tf.metrics.SparseCategoricalAccuracy())

In [63]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10
1016/1016 [==============================] - 78s 72ms/step - loss: 2.7344 - sparse_categorical_accuracy: 0.4828 - val_loss: 1.6094 - val_sparse_categorical_accuracy: 0.5486
Epoch 2/10
1016/1016 [==============================] - 77s 76ms/step - loss: 1.6095 - sparse_categorical_accuracy: 0.4691 - val_loss: 1.6094 - val_sparse_categorical_accuracy: 0.5489
Epoch 3/10
1016/1016 [==============================] - 80s 79ms/step - loss: 1.6095 - sparse_categorical_accuracy: 0.4691 - val_loss: 1.6094 - val_sparse_categorical_accuracy: 0.5489
Epoch 4/10
1016/1016 [==============================] - 76s 75ms/step - loss: 1.6095 - sparse_categorical_accuracy: 0.4691 - val_loss: 1.6094 - val_sparse_categorical_accuracy: 0.5489
Epoch 5/10
1016/1016 [==============================] - 71s 70ms/step - loss: 1.6095 - sparse_categorical_accuracy: 0.4691 - val_loss: 1.6094 - val_sparse_categorical_accuracy: 0.5489
Epoch 6/10
1016/1016 [==============================] - 71s 70ms/step - loss: 1.